In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt
import rioxarray
from carbonplan.data import cat
from carbonplan_styles.mpl import get_colormap
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable

from carbonplan_data import utils
alt.data_transformers.disable_max_rows()

In [ ]:
coarsen = 4
store = "az"
tlim = ("1984", "2018")

In [ ]:
historical_fire = xr.open_zarr(
    get_store(
        "carbonplan-forests", "risks/results/paper/fire_terraclimate_v6.zarr"
    )
)

In [ ]:
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [ ]:
historical_fire = (
    historical_fire.groupby("time.month")
    .mean()
    .where(fire_mask)
    .compute()["historical"]
)

In [ ]:
plot_params = {
    "cmap": get_colormap("reds"),
    "var_lims": (0, 0.0003),
    "label": "Burn area\n(fraction/year)",
    "y_loc": 0.2,
    "cbar_height": 0.6,
    "x_loc": 1.08,
    "width": 0.03,
}
matplotlib.rc("font", family="sans-serif")
matplotlib.rc("font", serif="Helvetica Neue")
matplotlib.rc("text", usetex="false")
matplotlib.rcParams.update({"font.size": 14, "svg.fonttype": "none"})

In [ ]:
months = ["Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov"]

In [ ]:
p = historical_fire.sel(month=slice(3, 11)).plot.imshow(
    col="month",
    col_wrap=3,
    vmin=0,
    vmax=0.001,
    subplot_kws={"projection": plot.cartopy_proj_albers()},
    add_colorbar=False,
    cmap=plot_params["cmap"],
    figsize=(8, 6),
)
for i, ax in enumerate(p.axes.flat):
    plot.map_pretty(ax, title=months[i])
    ax.set_title("")

plot.add_colorbar(
    p.fig,
    y_location=plot_params["y_loc"],
    vmin=plot_params["var_lims"][0],
    vmax=plot_params["var_lims"][1],
    cbar_label=plot_params["label"],
    cmap=plot_params["cmap"],
    height=plot_params["cbar_height"],
    x_location=plot_params["x_loc"],
    width=plot_params["width"],
)
plt.tight_layout(pad=-4)
plt.savefig("fig_supp7.svg", format="svg", bbox_inches="tight")